<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/pytorch_tutorials/pytorch_02%EF%BC%9A%E4%BD%BF%E7%94%A8%E5%9F%BA%E4%BA%8EAttention%E7%9A%84Seq2Seq%E7%BD%91%E7%BB%9C%E8%BF%9B%E8%A1%8C%E6%9C%BA%E5%99%A8%E7%BF%BB%E8%AF%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

&emsp;&emsp;Seq2Seq网络简单但功能强大，使用两个循环神经网络协同工作，将一个Sequence转换为另一个Sequence。其中，编码器网络将输入压缩为一个向量，而解码器将该向量展开为一个新序列。

<img src="https://pytorch.org/tutorials/_images/seq2seq.png" width="500">

&emsp;&emsp;为了改进上图所示的模型，将使用一种注意力机制，该机制使解码器学会专注于输入序列的特定范围。

&emsp;&emsp;首先导入所需的库

In [0]:
from __future__ import unicode_literals,print_function,division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
print(torch.__version__)

1.4.0


In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 1.加载数据文件、准备数据

&emsp;&emsp;使用的数据是英语到法语的翻译对集合。文件内容如下：

I am cold. \t J'ai froid.

&emsp;&emsp;与字符级RNN教程中使用的字符编码类似，我们将一种语言中的每个单词表示为一个one-hot向量。与语言中存在的几十个字符相比，单词要多的多，因此编码向量很大。但是我们将仅适用语言中的几千个单词。

<img src="https://pytorch.org/tutorials/_images/word-encoding.png" width="500">

&emsp;&emsp;每个单词都有唯一的索引，以便用作网络的输入和输出。因此将新建一个Lang类，该类具有单词->索引(word2index)和索引->单词(index2word)字典，以及每个单词word2count的计数，用于以后替换替换掉稀有单词。

In [0]:
SOS_token=0
EOS_token=1
class Lang:
  def __init__(self,name):
    self.name=name
    self.word2index={}
    self.word2count={}
    self.index2word={0:'SOS',1:"EOS"}
    self.n_words=2 #SOS和EOS
  def addSentence(self,sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self,word):
    if word not in self.word2index:
      self.word2index[word]=self.n_words
      self.word2count[word]=1
      self.index2word[self.n_words]=word
      self.n_words+=1
    else:
      self.word2count[word]+=1
      

&emsp;&emsp;文件都是Unicode编码，为简化起见，将Unicode编码转为ASCII，将所有内容都转为小写，并去掉大多数标点符号。

In [0]:
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFC',s)
      if unicodedata.category(c)!='Mn'
  )

def normalizeString(s):
  s=unicodeToAscii(s.lower().strip())#转为小写
  s=re.sub(r"([.!?])",r" \1",s)
  s=re.sub(r"[^a-zA-Z.!?]+",r" ",s)
  return s


&emsp;&emsp;要读取数据文件，我们将文件分成几行，然后将行分为两对。文件都是英语->其他语言的。因此，如果要从其他语言->英语翻译，添加翻转标记以反转语句对。

In [0]:
def readLangs(lang1,lang2,base_path,reverse=False):
  print('Reading lines...')

  lines=open(base_path+"%s-%s-small.txt"%(lang1,lang2),encoding='utf-8').read().strip().split('\n')
  print(len(lines))
  pairs=[[normalizeString(s) for s in l.split('\t')] for l in lines]

  if reverse:
    pairs=[list(reversed(p)) for p in pairs]
    input_lang=Lang(lang2)
    output_lang=Lang(lang1)
  else:
    input_lang=Lang(lang1)
    output_lang=Lang(lang2)

  return input_lang,output_lang,pairs

&emsp;&emsp;由于语句对比较多，但是想训练的快速一点。因此将数据集修剪为相对较短和简单的句子。最大长度为10个字(包括结尾的标点符号)，并且过滤掉翻译成“我是”或“他是”等形式的句子。

In [0]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
  return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH #and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

完整的数据预处理流程如下：
* 读取文本文件，并划分为行和语句对。
* 标准化文本，并对文本进行过滤
* 构造单词列表

In [0]:
def prepareData(lang1,lang2,base_path,reverse=False):
  input_lang,output_lang,pairs=readLangs(lang1,lang2,base_path,reverse)
  print("Read %s sentence pairs"%len(pairs))
  pairs=filterPairs(pairs)
  print("Trimmed to %s sentence pairs"%len(pairs))
  print("Counting words...")
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
  print("Counted words:")
  print(input_lang.name,input_lang.n_words)
  print(output_lang.name,output_lang.n_words)
  return input_lang,output_lang,pairs

In [9]:
!ls /content/drive/My\ Drive/data/d2l-zh-tensoflow/

airfoil_self_noise.dat	img			ptb
fr-en-small.txt		jaychou_lyrics.txt.zip	ptb.zip


In [10]:
base_path="/content/drive/My Drive/data/d2l-zh-tensoflow/"
input_lang,output_lang,pairs=prepareData('fr','en',base_path,True)
print(random.choice(pairs))

Reading lines...
20
Read 20 sentence pairs
Trimmed to 20 sentence pairs
Counting words...
Counted words:
en 37
fr 45
['he is my uncle .', 'c est mon oncle .']


In [11]:
len(pairs)

20

## 2. Seq2Seq模型

&emsp;&emsp;循环神经网络(RNN)使用自己输出作为下一步的输入。Seq2Seq网络也称为Encoder-Decoder网络，是被称为encoder和decoder的两个RNN网络组成的模型。Encoder读取输入然后输出单个向量，Decoder读取该向量并产生一个输出序列。

<img src="https://pytorch.org/tutorials/_images/seq2seq.png" width="500">

&emsp;&emsp;不像使用单个RNN进行序列预测(每个输入对应一个输出),seq2seq模型使我们摆脱了序列长度和顺序的限制，这使其非常适合在两种语言之间进行翻译。

&emsp;&emsp;考虑序列“Je ne suis pas le chat noir” → “I am not the black cat”. 输入序列中的大多数单词在输出序列中有着直接的翻译，但是顺序略有不同，例如“chat noir” and “black cat”.由于采用"ne/pas"结构，因此在输入句子中还有一个单词。直接从输入单词的序列中产生正确的单词比较困难。

&emsp;&emsp;使用seq2seq模型，编码器会创建一个向量，在理想情况下，该向量将输入序列的“含义”编码为一个向量，能够充分表达序列的意思。

### 2.1 编码器

&emsp;&emsp;seq2seq网络的编码器是一个RNN网络，它为输入句子中的每个单词输出一些值，对于每个输入字，编码器输出一个向量和一个隐藏状态，并将隐藏状态用于下一个输入字。

<img src="https://pytorch.org/tutorials/_images/encoder-network.png" width="300">

In [0]:
class EncoderRNN(nn.Module):
  def __init__(self,input_size,hidden_size):
    super(EncoderRNN,self).__init__()
    self.hidden_size=hidden_size

    self.embedding=nn.Embedding(input_size,hidden_size)
    self.gru=nn.GRU(hidden_size,hidden_size)

  def forward(self,input,hidden):
    embedded=self.embedding(input).view(1,1,-1)
    output=embedded
    output,hidden=self.gru(output,hidden)
    return output,hidden

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)

## 2.2 解码器
&emsp;&emsp;解码器是另一个RNN网络，使用编码器的输出向量作为输入，然后输出一个单词序列进行翻译任务。

### 2.2.1 简单解码器
&emsp;&emsp;在简单解码器中，我们仅使用编码器最后时间步的输出作为输入。最后时间步的输出有时被称为上下文向量，因为它编码了整个序列的上下文。这个上下文向量被用作解码器的初始隐藏状态。
&emsp;&emsp;在每个时间步进行解码时，解码器被给予一个token和隐藏状态作为输入。初始的token是字符序列的起始token <SOS>,初始隐藏状态是上下文向量(编码器的最后隐藏状态)。

<img src="https://pytorch.org/tutorials/_images/decoder-network.png" width="300">


In [0]:
class DecoderRNN(nn.Module):
  def __init__(self,hidden_size,output_size):
    super(DecoderRNN,self).__init__()
    self.hidden_size=hidden_size

    self.embedding=nn.Embedding(output_size,hidden_size)
    self.gru=nn.GRU(hidden_size,hidden_size)
    self.out=nn.Linear(hidden_size,output_size)
    self.softmax=nn.LogSoftmax(dim=1)

  def forward(self,input,hidden):
    output=self.embedding(input).view(1,1,-1)
    output=F.relu(output)
    output,hidden=self.gru(output,hidden)
    output=self.softmax(self.out(output[0]))
    return output,hidden

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)

### 2.2.2 注意力机制的Decoder
&emsp;&emsp;如果只有上下文向量在编码器和解码器之间传递，则该单个向量承担对整个句子进行编码的负担。注意力机制使解码器可以针对解码器自身输出的每一步，专注于编码器输出的不同部分。首先，我们计算一组注意力权重，将这些权重与编码器输出向量相乘以创建加权组合。结果(在代码中称为attn_applied)应包含有关输入序列特定部分的信息，从而帮助解码器选择正确的输出单词。

<img src="https://i.imgur.com/1152PYf.png" width="300">

&emsp;&emsp;注意力的权重使用另一个全连接层 attn进行计算，使用decoder的输入和隐藏层作为输入。由于训练数据中包含各种长度的句子，因此要实际创建和训练该层，我们必须选择可以应用的最大句子长度(输入长度，用于编码器输出).最大长度的句子将使用所有注意力权重，而较短的句子将仅使用前几个。

<img src="https://pytorch.org/tutorials/_images/attention-decoder-network.png" width="300">



In [0]:
class AttnDecoderRNN(nn.Module):
  def __init__(self,hidden_size,output_size,dropout_p=0.1,max_length=MAX_LENGTH):
    super(AttnDecoderRNN,self).__init__()
    self.hidden_size=hidden_size
    self.output_size=output_size
    self.dropout_p=dropout_p
    self.max_length=max_length

    self.embedding=nn.Embedding(self.output_size,self.hidden_size)
    self.attn=nn.Linear(self.hidden_size*2,self.max_length)
    self.attn_combine=nn.Linear(self.hidden_size*2,self.hidden_size)
    self.dropout=nn.Dropout(self.dropout_p)
    self.gru=nn.GRU(self.hidden_size,self.hidden_size)
    self.out=nn.Linear(self.hidden_size,self.output_size)

  def forward(self,input,hidden,encoder_outputs):
    embedded=self.embedding(input).view(1,1,-1)
    embedded=self.dropout(embedded)

    attn_weights=F.softmax(self.attn(torch.cat((embedded[0],hidden[0]),1)),dim=1)

    attn_applied=torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))

    output=torch.cat((embedded[0],attn_applied[0]),1)
    output=self.attn_combine(output).unsqueeze(0)
    output=F.relu(output)
    output,hidden=self.gru(output,hidden)
    output=F.log_softmax(self.out(output[0]),dim=1)
    return output,hidden,attn_weights

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)


## 3.  训练

### 3.1 准备训练数据
&emsp;&emsp;为了训练，对于每个语句对，我们需要一个输入张量(输入序列的索引)和目标张量(目标序列的词索引)。需要在两个序列后添加EOS token。

In [0]:
def indexesFromSentence(lang,sentence):
  return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang,sentence):
  indexes=indexesFromSentence(lang,sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes,dtype=torch.long,device=device).view(-1,1)

def tensorsFromPairs(pair):
  input_tensor=tensorFromSentence(input_lang,pair[0])
  target_tensor=tensorFromSentence(output_lang,pair[1])
  return (input_tensor,target_tensor)

### 3.2 训练模型
&emsp;&emsp;为了进行训练，我们通过编码器运行输入语句，并跟踪每个输出和最新的隐藏状态。然后，为解码器提供<SOS>token作为第一个输入，并将编码器的最后一个隐藏状态作为其第一个隐藏状态。

&emsp;&emsp;"Teacher forcing"是使用真实的输出作为下一个输入，而不是使用编码器的预测作为输入。使用"Teacher forcing"会使训练收敛的更快，但是使用网络时可能会出现不稳定。



In [0]:
teacher_forcing_ratio=0.5

def train(input_tensor,target_tensor,encoder,decoder,encoder_optimizer,decoder_optimizer,criterion,max_length=MAX_LENGTH):
  encoder_hidden=encoder.initHidden()

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  input_length=input_tensor.size(0)
  target_length=target_tensor.size(0)

  encoder_outputs=torch.zeros(max_length,encoder.hidden_size,device=device)

  loss=0

  for ei in range(input_length):
    encoder_output,encoder_hidden=encoder(input_tensor[ei],encoder_hidden)
    encoder_outputs[ei]=encoder_output[0,0]

  decoder_input=torch.tensor([[SOS_token]],device=device) #SOS token作为解码器的初始输入

  decoder_hidden=encoder_hidden #编码器的最后时间步的隐藏状态作为解码器的初始隐藏状态

  use_teacher_forcing=True if random.random() <teacher_forcing_ratio else False

  if use_teacher_forcing:
    for di in range(target_length):
      decoder_output,decoder_hidden,decoder_attention=decoder(decoder_input,decoder_hidden,encoder_outputs)
      loss+=criterion(decoder_output,target_tensor[di])
      decoder_input=target_tensor[di]

  else:
    for di in range(target_length):
      decoder_output,decoder_hidden,decoder_attention=decoder(decoder_input,decoder_hidden,encoder_outputs)
      topv,topi=decoder_output.topk(1)
      decoder_input=topi.squeeze().detach() #detach from history as input
      loss+=criterion(decoder_output,target_tensor[di])
      if decoder_input.item()==EOS_token:
        break

  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item()/target_length


In [0]:
import time
import math

In [0]:
def asMinutes(s):
  m=math.floor(s/60)
  s-=m*60
  return '%dm %ds'%(m,s)

def timeSince(since,percent):
  now=time.time()
  s=now-since 
  es=s/(since)
  rs=es-s
  return '%s(- %s)'%(asMinutes(s),asMinutes(rs))

### 整个训练过程包括

* 启动一个计时器
* 初始化优化器和损失函数
* 创建训练数据集
* 创建空损失数组用于绘图

In [0]:
def trainIters(encoder,decoder,n_iters,print_every=1000,plot_every=100,learning_rate=0.01):
  start=time.time()
  plot_losses=[]
  print_loss_total=0
  plot_loss_total=0
  encoder_optimizer=optim.SGD(encoder.parameters(),lr=learning_rate)
  decoder_optimizer=optim.SGD(encoder.parameters(),lr=learning_rate)
  training_pairs=[tensorsFromPairs(random.choice(pairs)) for i in range(n_iters)]
  criterion=nn.NLLLoss()
  for iter in range(1,n_iters+1):
    training_pair=training_pairs[iter-1]
    input_tensor=training_pair[0]
    target_tensor=training_pair[1]
    loss=train(input_tensor,target_tensor,encoder,decoder,encoder_optimizer,decoder_optimizer,criterion)
    print_loss_total+=loss
    plot_loss_total+=loss

    if iter % print_every==0:
      print_loss_avg=print_loss_total / print_every
      print_loss_total=0
      print("%s (%d %d%%) %.4f"%(timeSince(start,iter/n_iters),iter,iter/n_iters*100,print_loss_avg))
    if iter % plot_every==0:
      plot_loss_avg=plot_loss_total/plot_every
      plot_losses.append(plot_loss_avg)
      plot_loss_total=0
  showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

In [0]:
def showPlot(points):
  plt.figure()
  fig,ax=plt.subplots()
  loc=ticker.MultipleLocator(base=0.2)
  ax.yaxis.set_major_locator(loc)
  plt.plot(points)

In [0]:
def evaluate(encoder,decoder,sentence,max_length=MAX_LENGTH):
  with torch.no_grad():
    input_tensor=tensorFromSentence(input_lang,sentence)
    input_length=input_tensor.size()[0]
    encoder_hidden=encoder.initHidden()
    encoder_outputs=torch.zeros(max_length,encoder.hidden_size,device=device)
    for ei in range(input_length):
      encoder_output,encoder_hidden=encoder(input_tensor[ei],encoder_hidden)
      encoder_outputs[ei]+=encoder_output[0,0]
    decoder_input=torch.tensor([[SOS_token]],device=device)
    decoder_hidden=encoder_hidden
    decoded_words=[]
    decoder_attentions=torch.zeros(max_length,max_length)

    for di in range(max_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topk(1)
      if topi.item() == EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[topi.item()])

      decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [25]:
hidden_size=256
encoder1=EncoderRNN(input_lang.n_words,hidden_size).to(device)
attn_decoder1=AttnDecoderRNN(hidden_size,output_lang.n_words,dropout_p=0.1).to(device)
trainIters(encoder1,attn_decoder1,75000,print_every=5000)

1m 14s(- -2m 45s) (5000 6%) 3.1094
2m 27s(- -3m 32s) (10000 13%) 2.9876
3m 41s(- -4m 18s) (15000 20%) 2.9589
4m 57s(- -5m 2s) (20000 26%) 2.9449
6m 13s(- -7m 46s) (25000 33%) 2.9438
7m 30s(- -8m 29s) (30000 40%) 2.9249
8m 45s(- -9m 14s) (35000 46%) 2.9264
10m 1s(- -11m 58s) (40000 53%) 2.9190
11m 18s(- -12m 41s) (45000 60%) 2.9218
12m 35s(- -13m 24s) (50000 66%) 2.9173
13m 52s(- -14m 7s) (55000 73%) 2.9173
15m 5s(- -16m 54s) (60000 80%) 2.9173
16m 21s(- -17m 38s) (65000 86%) 2.9188
17m 34s(- -18m 25s) (70000 93%) 2.9171
18m 47s(- -19m 12s) (75000 100%) 2.9178
